In [2]:
import pandas as pd
import altair as alt
df_total=pd.read_csv('Datos_total.csv')
df_apertura=pd.read_csv('Datos_apertura.csv')
df_clausura=pd.read_csv('Datos_clausura.csv')
import duckdb


In [95]:
#pasar todo a int
nombres=['Datos_apertura.csv','Datos_clausura.csv']
tablas=[df_apertura,df_clausura]
columnas=['Partidos Jugados','Titular','Suplente','Sin ingresar','G + A','Goles','Asistencias','T.A','T.R','Minutos de titular','Minutos de Suplente','Total']
for tabla in range(len(tablas)):
    for columna in columnas:
        tablas[tabla][columna]=tablas[tabla][columna].astype(int)#Partidos Jugados,Titular,Suplente,Sin ingresar,G + A,Goles,Asistencias,T.A,T.R,Minutos de titular,Minutos de Suplente,Total
    tablas[tabla].to_csv(nombres[tabla],index=False)

In [96]:
# Grafico 1 partidos jugados vs partidos citados
df_apertura1= duckdb.sql('SELECT "Jugador","Partidos Jugados","partidos Jugados" + "sin ingresar" AS "Partidos citados" FROM df_apertura').to_df()
df_clausura1=duckdb.sql('SELECT "Jugador","Partidos Jugados","partidos Jugados" + "sin ingresar" AS "Partidos citados" FROM df_clausura').to_df()
df_total1=duckdb.sql('SELECT "Jugador","Partidos Jugados","partidos Jugados" + "sin ingresar" AS "Partidos citados" FROM df_total').to_df()


def partidos_citados(nombre_sql,titulo):
    bar = alt.Chart(nombre_sql).mark_bar().encode(
        x=alt.X('Partidos Jugados:Q',sort=None,title='Partidos Jugados / Partidos Citados'),
        y=alt.Y('Jugador:N',sort='-x',title=None),
        tooltip= [alt.Tooltip('Partidos citados'),alt.Tooltip('Partidos Jugados')],
        
    ).properties(
        width=400, 
        height=500,
        title= titulo
    )

    tick = alt.Chart(nombre_sql).mark_tick(
        color='red',
        thickness=2,
        size=20 * 0.9,  # controls width of tick.
    ).encode(
        x=alt.X('Partidos citados:Q'),
        y=alt.Y('Jugador:N',sort='-x')
    )
  

    grafico=bar+tick
    return grafico

grafico_apertura=partidos_citados(df_apertura1,'Apertura')
grafico_clausura=partidos_citados(df_clausura1,'Clausura')
grafico_total=partidos_citados(df_total1,'Total')


grafico1=grafico_apertura|grafico_clausura|grafico_total





In [98]:
# Grafico 2 Partidos jugados (suplentes) VS partidos citados
df_apertura1= duckdb.sql('SELECT "Jugador","Suplente","Suplente" + "sin ingresar" AS "Partidos citados suplente" FROM df_apertura WHERE "Suplente">0').to_df()
df_clausura1=duckdb.sql('SELECT "Jugador","Suplente","Suplente" + "sin ingresar" AS "Partidos citados suplente" FROM df_clausura WHERE "Suplente">0').to_df()
df_total1=duckdb.sql('SELECT "Jugador","Suplente","Suplente" + "sin ingresar" AS "Partidos citados suplente" FROM df_total WHERE "Suplente">0').to_df()


def partidos_citados_suplente(nombre_sql,titulo):
    bar = alt.Chart(nombre_sql).mark_bar().encode(
        x=alt.X('Suplente',sort=None,title='Partidos como suplente / Partidos Citados'),
        y=alt.Y('Jugador:N',sort='-x',title=None),
        tooltip= [alt.Tooltip('Suplente'),alt.Tooltip('Partidos citados suplente')],
        
    ).properties(
        width=400, 
        height=500,  
        title= titulo
    )

    tick = alt.Chart(nombre_sql).mark_tick(
        color='red',
        thickness=2,
        size=20 * 0.9,  # controls width of tick.
    ).encode(
        x=alt.X('Partidos citados suplente:Q'),
        y=alt.Y('Jugador:N',sort='-x')
    )
  

    grafico=bar+tick
    return grafico

grafico_apertura=partidos_citados_suplente(df_apertura1,'Apertura')
grafico_clausura=partidos_citados_suplente(df_clausura1,'Clausura')
grafico_total=partidos_citados_suplente(df_total1,'Total')


grafico2=grafico_apertura|grafico_clausura|grafico_total
grafico2


 

alt.HConcatChart(...)

In [114]:
#3 Minutos jugados como Suplente
df_apertura1= duckdb.sql('SELECT "Jugador","Suplente","Minutos de Suplente" FROM df_apertura WHERE "Suplente">0').to_df()
df_clausura1=duckdb.sql('SELECT "Jugador","Suplente","Minutos de Suplente" FROM df_clausura WHERE "Suplente">0').to_df()
df_total1=duckdb.sql('SELECT "Jugador","Suplente","Minutos de Suplente" FROM df_total WHERE "Suplente">0').to_df()

def boxplot1(data, titulo):  # MINUTOS DE SUPLENTE/SUPLENTE
    a = 0
    if data['Minutos de Suplente'].max() >= 300:
        b = 350
    else:
        b = 240

    # Crear la selección para filtrar por jugador con un menú desplegable

    # Gráfico de puntos con el filtro
    points = alt.Chart(data).mark_point(size=20,color='red').encode(
        alt.Y('Minutos de Suplente', scale=alt.Scale(domain=[a, b])),
        tooltip=[alt.Tooltip('Jugador'), alt.Tooltip('Minutos de Suplente'), alt.Tooltip('Suplente')],
    ).properties(
        width=200,
        height=500
    ).interactive()  # Hacer que el gráfico sea interactivo

    # Gráfico de boxplot
    boxplot = alt.Chart(data).mark_boxplot().encode(
        alt.Y('Minutos de Suplente', scale=alt.Scale(domain=[a, b])),
    ).properties(
        width=200,
        height=500,
        title=titulo
    )

    combined_chart = boxplot + points
    return combined_chart

grafico_apertura=boxplot1(df_apertura1,'Apertura')
grafico_clausura=boxplot1(df_clausura1,'Clausura')
grafico_total=boxplot1(df_total1,'Total')

grafico5=grafico_apertura|grafico_clausura|grafico_total
grafico5





alt.HConcatChart(...)

In [115]:
# 4 Minutos por partido como suplente jugadores que entraron mas de una vez
# otra forma de agregar otra columna usando pandas 
df_total1=df_total
df_total1['Minutos x partido suplente']= df_total1['Minutos de Suplente'] / df_total1['Suplente'] 
df_total1=df_total1[df_total1['Minutos x partido suplente']>0 ] #filtrando datos
df_total1=df_total1[df_total1['Suplente']>1] # mas facil hacerlo con sql

df_apertura1=df_apertura
df_apertura1['Minutos x partido suplente']= df_apertura1['Minutos de Suplente'] / df_apertura1['Suplente'] 
df_apertura1=df_apertura1[df_apertura1['Minutos x partido suplente']>0 ] #filtrando datos
df_apertura1=df_apertura1[df_apertura1['Suplente']>1] # mas facil hacerlo con sql

df_clausura1=df_clausura
df_clausura1['Minutos x partido suplente']= df_clausura1['Minutos de Suplente'] / df_clausura1['Suplente'] 
df_clausura1=df_clausura1[df_clausura1['Minutos x partido suplente']>0 ] #filtrando datos
df_clausura1=df_clausura1[df_clausura1['Suplente']>1] # mas facil hacerlo con sql

def boxplot2(data, titulo):
    # Crear la selección para filtrar por jugador con un menú desplegable

    # Gráfico de puntos con el filtro
    points = alt.Chart(data).mark_point(size=20,color='red').encode(
        alt.Y('Minutos x partido suplente', scale=alt.Scale(domain=[0, 40])),
        tooltip=[alt.Tooltip('Jugador'), alt.Tooltip('Minutos x partido suplente'), alt.Tooltip('Suplente')],
        ).properties(
        width=200,
        height=500
    ).interactive()  # Hacer que el gráfico sea interactivo

    # Gráfico de boxplot
    boxplot = alt.Chart(data).mark_boxplot().encode(
        alt.Y('Minutos x partido suplente'),
    ).properties(
        width=200,
        height=500,
        title=titulo
    )

    # Combinar ambos gráficos
    combined_chart = boxplot + points
    return combined_chart
grafico_apertura=boxplot2(df_apertura1,'Apertura')
grafico_clausura=boxplot2(df_clausura1,'Clausura')
grafico_total=boxplot2(df_total1,'Total')

grafico6=grafico_apertura|grafico_clausura|grafico_total
grafico6



alt.HConcatChart(...)

In [117]:
# 5 minutos jugados como titular x partido jugando mas de 1 vez
# use otro porque en este se pasa de 90, el otro estaba muy apretado
df_apertura1=duckdb.sql('''SELECT "Jugador","Titular","Minutos de titular","Minutos de titular"/"Titular" AS "Minutos como titular"
                        FROM "df_apertura" 
                        WHERE "Titular">1
                        ''').to_df()

df_clausura1=duckdb.sql('''SELECT "Jugador","Titular","Minutos de titular","Minutos de titular"/"Titular" AS "Minutos como titular"
                        FROM "df_clausura" 
                        WHERE "Titular">1
                        ''').to_df()
df_total1=duckdb.sql('''SELECT "Jugador","Titular","Minutos de titular","Minutos de titular"/"Titular" AS "Minutos como titular"
                        FROM "df_total" 
                        WHERE "Titular">1
                        ''').to_df()

def boxplot3(data, titulo):
    # Crear la selección para filtrar por jugador con un menú desplegable

    # Gráfico de puntos con el filtro
    points = alt.Chart(data).mark_point(size=20,color='red').encode(
        alt.Y('Minutos como titular', scale=alt.Scale(domain=[50, 95])),
        tooltip=[alt.Tooltip('Jugador'), alt.Tooltip('Minutos como titular'), alt.Tooltip('Minutos de titular'), alt.Tooltip('Titular')],
        ).properties(
        width=200,
        height=500
    ).interactive()  # Hacer que el gráfico sea interactivo

    # Gráfico de boxplot
    boxplot = alt.Chart(data).mark_boxplot().encode(
        alt.Y('Minutos como titular:Q', scale=alt.Scale(domain=[50, 95])),
    ).properties(
        width=200,
        height=500,
        title=titulo
    )

    # Combinar ambos gráficos
    combined_chart = boxplot + points
    return combined_chart

grafico_apertura=boxplot3(df_apertura1,'Apertura')
grafico_clausura=boxplot3(df_clausura1,'Clausura')
grafico_total=boxplot3(df_total1,'Total')

grafico7=grafico_apertura|grafico_clausura|grafico_total
grafico7



alt.HConcatChart(...)

In [120]:

#6 minutos jugados en todo el anio

def boxplot4(data, titulo):
    a=-5
    if data['Total'].max()>1170:
        b=2400
    else:
        b=1200
    # Crear la selección para filtrar por jugador con un menú desplegable

    # Gráfico de puntos con el filtro
    points = alt.Chart(data).mark_point(size=20,color='red').encode(
        alt.Y('Total:Q',scale=alt.Scale(domain=[a,b])),
        tooltip=[alt.Tooltip('Jugador'), alt.Tooltip('Total:Q')],
    ).properties(
        width=200,
        height=500
    ).interactive()  # Hacer que el gráfico sea interactivo

    # Gráfico de boxplot
    boxplot = alt.Chart(data).mark_boxplot().encode(
        alt.Y('Total:Q'),
    ).properties(
        width=200,
        height=500,
        title=titulo
    )  

    # Combinar ambos gráficos
    combined_chart = boxplot + points
    return combined_chart

grafico_apertura=boxplot4(df_apertura,'Apertura')
grafico_clausura=boxplot4(df_clausura,'Clausura')
grafico_total=boxplot4(df_total,'Total')

grafico8=grafico_apertura|grafico_clausura|grafico_total
grafico8


alt.HConcatChart(...)

In [17]:
#7 Histograma de edades
df_total1 = duckdb.sql('SELECT  count(*) AS "Jugadores por edad","Edad" FROM "df_total" GROUP BY "Edad"' ).to_df()
def histograma(data,x,y,titulo,tooltip):
    if tooltip:
        histograma_edad = alt.Chart(data).mark_bar().encode(
            alt.X(x, bin=alt.Bin(extent=[17, 27], step=1),title='Edad'),  # Define el rango y el tamaño del bin
            alt.Y(y,title=titulo),
            tooltip=[alt.Tooltip(y), alt.Tooltip(tooltip)]
        ).properties(
            width=400,
            height=400,
        )
    else:
        histograma_edad = alt.Chart(data).mark_bar().encode(
            alt.X(x, bin=alt.Bin(extent=[17, 27], step=1),title='Edad'),  # Define el rango y el tamaño del bin
            alt.Y(y,title=titulo),
            tooltip=[alt.Tooltip(y)]
        ).properties(
            width=400,
            height=400,
        
        )
        
    return histograma_edad
minutos_total=histograma(df_total,'Edad','Total','Minutos Jugados','Jugador:N')
personas_total=histograma(df_total1,'Edad','Jugadores por edad','Cantidad de jugadores por edad',0)
grafico9=personas_total|minutos_total
grafico9



alt.HConcatChart(...)

In [3]:
#8 EDAD VS MINUTOS JUGADOS 
# NO HAY CORRELACION


source = df_total

# Crear el gráfico de dispersión
graph = alt.Chart(source).mark_circle(size=60).encode(
    alt.X('Edad', scale=alt.Scale(domain=[15, 30]), title='Edad'),
    y=alt.Y('Total', title='Total'),
    tooltip=['Edad', 'Total', 'Jugador']
).interactive().properties(
    width=500,
    title='Relación entre Edad y Total'
)

# Añadir la línea de regresión
linea_regresion = graph.transform_regression(
    'Edad', 'Total'
).mark_line(color='red')


# Calcular la correlación
correlacion = df_total['Total'].corr(df_total['Edad'])
print(f'Correlación: {correlacion}')

# Combinar el gráfico de dispersión y la línea de regresión
grafico10 = graph + linea_regresion 
grafico10



Correlación: 0.10686835060984036


alt.LayerChart(...)

In [14]:
import statsmodels.api as sm
import numpy as np

# Supongamos que tienes datos de X (variables independientes) y Y (variable dependiente)
X = df_total['Edad'].values.reshape(-1, 1)  # Convertir en vector columna
Y = df_total['Total'].values  # Dejar Y como un vector

# Agregar un intercepto a X
X = sm.add_constant(X)

# Ajustar el modelo de regresión lineal
model = sm.OLS(Y, X).fit()

# Obtener un resumen de los resultados
print(model.summary())



                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                 -0.023
Method:                 Least Squares   F-statistic:                    0.3350
Date:                Mon, 21 Oct 2024   Prob (F-statistic):              0.567
Time:                        17:05:43   Log-Likelihood:                -246.85
No. Observations:                  31   AIC:                             497.7
Df Residuals:                      29   BIC:                             500.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        178.2162   1126.785      0.158      0.8

In [123]:

#9 
#Minutos jugados por posicion
data = duckdb.sql('''SELECT "Jugador", "Total", "Posicion" FROM "df_total" 
                ORDER BY 
                "Posicion",
                "Total" DESC;''').to_df()

pts = alt.selection_point(encodings=['x'])

# Especificar el orden de las posiciones

# Gráfico de barras para los minutos jugados por posición
bar_chart = alt.Chart(data).mark_bar().encode(
    x=alt.X('Posicion:N', title='Posición'),  # Orden personalizado de las posiciones
    y=alt.Y('Total:Q', title='Minutos jugados', sort='-x'),  # Orden de menor a mayor minutos jugados
    color=alt.condition(pts, alt.ColorValue("orange"), alt.ColorValue("grey")),
    tooltip=[alt.Tooltip('Posicion', title='Posición'), alt.Tooltip('Total:Q', title='Minutos jugados'), alt.Tooltip('Jugador', title='Jugador')]
).add_params(
    pts
).properties(
    width=500,
    height=300,
    title="Minutos jugados por posición"
)

# Gráfico de barras para la cantidad de jugadores por posición
genre_bar = alt.Chart(data, width=550, height=300).mark_bar().encode(
    x=alt.X('Posicion:N', title='Posición'),
    y=alt.Y('count()', title='Cantidad de jugadores'),
    color=alt.condition(pts, alt.ColorValue("steelblue"), alt.ColorValue("grey")),
    tooltip=[alt.Tooltip('Posicion', title='Posición'), alt.Tooltip('count()', title='Cantidad de jugadores')]
).add_params(
    pts
).properties(
    title="Cantidad de jugadores por posición (Interactivo)"
)

# Combinar ambos gráficos
grafico11=alt.hconcat(
    genre_bar,bar_chart
).resolve_legend(
    color="independent"
)





In [127]:
#10
data_total=duckdb.sql('''SELECT "Jugador","G/A cada 90","Goles","Asistencias","Total","Partidos Jugados" FROM df_total WHERE "G/A cada 90">0''').to_df()
data_apertura=duckdb.sql('''SELECT "Jugador","G/A cada 90","Goles","Asistencias","Total","Partidos Jugados" FROM df_apertura WHERE "G/A cada 90">0''').to_df()
data_clausura=duckdb.sql('''SELECT "Jugador","G/A cada 90","Goles","Asistencias","Total","Partidos Jugados" FROM df_clausura WHERE "G/A cada 90">0''').to_df()
def goles(data,titulo):
    base = alt.Chart(data).encode(
        x=alt.X("G/A cada 90",title='G/A cada 90'),
        y=alt.X("Jugador:O",sort='-x'),
        text="G/A cada 90",
        tooltip=[alt.Tooltip('Partidos Jugados'),alt.Tooltip('Total',title='Minutos jugados'),alt.Tooltip('Goles'),alt.Tooltip('Asistencias')]
    ).properties(
        title=titulo
    )
    graph=base.mark_bar() + base.mark_text(align='left', dx=2)
    return graph
graph3=goles(data_total,'Total')
graph1=goles(data_apertura,'Apertura')
graph2=goles(data_clausura,'Clausura')
grafico3=graph1|graph2|graph3



In [128]:
#11
data1=duckdb.sql('''SELECT "Jugador","Total","T.A","T.R",round(("T.A"+"T.R") * 90 /"Total",2) AS "T cada 90" FROM df_apertura WHERE "T.A">0
                ''').to_df()
data2=duckdb.sql('''SELECT "Jugador","Total","T.A","T.R",round(("T.A"+"T.R")* 90 /"Total",2) AS "T cada 90" FROM df_clausura WHERE "T.A">0
                ''').to_df()
data3=duckdb.sql('''SELECT "Jugador","Total","T.A","T.R",round(("T.A"+"T.R")* 90 /"Total",2) AS "T cada 90" FROM df_total WHERE "T.A">0
                ''').to_df()

def amarillas(data,titulo):
    base = alt.Chart(data).encode(
        x=alt.X("T cada 90",title='T cada 90'),
        y=alt.X("Jugador:O",sort='-x'),
        text="T cada 90",
        tooltip=[alt.Tooltip('Total',title='Minutos jugados')]
    ).properties(
        title=titulo
    )
    graph=base.mark_bar() + base.mark_text(align='left', dx=2)
    return graph
graph1=amarillas(data1,'Aperutra')
graph2=amarillas(data2,'Clausura')
graph3=amarillas(data3,'Total')
grafico4=graph1|graph2|graph3



In [129]:
chart = alt.vconcat(
    grafico1,
    grafico2,
    grafico3,
    grafico4,
    grafico5,
    grafico6,
    grafico7,
    grafico8,
    grafico9,
    grafico10,
    grafico11
)

chart.save(
    'Graficos Primera D.html',
    embed_options={'renderer':'svg','actions':False,'inline':True})

In [11]:
df_total.describe()
#df_total['Posicion'].nunique()
#df_total['Posicion'].value_counts()
#df_total.columns


,Partidos Jugados,Titular,Suplente,Sin ingresar,G + A,Goles,Asistencias,T.A,T.R,Minutos de titular,Minutos de Suplente,Total,Goles cada 90,Asistencias cada 90,G/A cada 90,Edad
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,12.774194,9.225806,3.548387,3.129032,1.161290,0.290323,0.870968,2.096774,0.225806,741.903226,84.225806,826.129032,0.094839,0.104516,0.198710,22.451613
std,8.106416,8.425001,3.731297,4.372200,5.342264,4.824891,1.024433,2.150269,0.616964,727.872304,93.786179,710.483391,0.310450,0.141653,0.377392,2.631080
min,1.000000,0.000000,0.000000,0.000000,-22.000000,-22.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,-0.870000,0.000000,-0.870000,17.000000
25%,6.500000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,119.500000,6.000000,241.000000,0.000000,0.000000,0.000000,20.500000
50%,14.000000,5.000000,3.000000,2.000000,1.000000,0.000000,1.000000,2.000000,0.000000,393.000000,61.000000,590.000000,0.000000,0.050000,0.100000,23.000000
75%,18.500000,13.000000,5.000000,3.500000,3.000000,1.000000,1.500000,3.500000,0.000000,1132.000000,106.500000,1193.000000,0.080000,0.160000,0.290000,25.000000
max,26.000000,26.000000,14.000000,21.000000,16.000000,13.000000,3.000000,7.000000,3.000000,2336.000000,322.000000,2336.000000,0.970000,0.510000,1.300000,26.000000
